# Problem Statement:

Write a function that takes the input, gives the output, and meets the conditions below: 

- Input: An N x M matrix of a garden. Each cell contains an integer representing the number of carrots in that part of the garden.

- Output: The number of carrots Bunny eats before falling asleep.

- Conditions: Bunny starts in the center of the garden. If there are more than one center cell, Bunny starts in the cell with the largest number of carrots. There will never be a tie for the highest number of carrots in a center cell. Bunny eats all of the carrots in his cell, then looks left, right, up, and down for more carrots. Bunny always moves to the adjacent cell with the highest carrot count. If there are no adjacent cells with carrots, Bunny falls asleep.

Example test case in python: 
garden1 = [[5, 7, 8, 6, 3],
               [0, 0, 7, 0, 4],
               [4, 6, 3, 4, 9],
               [3, 1, 0, 5, 8]]

eat(garden1)
27  # starts at garden[1][2] = 7, eats 7 carrots, looks at the 8, 0, 3, and 0 adjacent, moves to the 8, repeat.



# Approach Overview:

There's a few different challenges going on in this game. First, we need to identify the center of the board, which we will do with a function that we will call once. Second, we need to look at the carrot counts for all the squares around Bunny. This is a maximum of 4 cells, for central squares, or a minimum of 0 cells, for empty gardens (input garden = [ ]). Third, we need to update the garden with the carrots as Bunny moves around, and update the carrot count as he eats the carrots.

I solved this problem with 4 functions. The function find_center() finds the center. The function compare_coordinates() determines the direction to move, and calls another function, check_positions() to make sure those directions are within the garden. The final eat() function then calls find_center() and compare_coordinates, checking at each loop to make sure the bunny is awake.

# Handling Edge Cases:
- Empty garden will throw an error (cannot index with find_center()) however one more line could be added to handle this exception.
- Negative carrots should be handled fine.
- Non-numeric values: will throw an error.
- Rows with different numbers of cells compared to the first row: this currently cannot be handled, but the problem statement excludes this scenario.
- Ties: Bunny will select the highest carrot in this order of priority: top, bottom, left, right. No information on priorities was provided, but the bunny needs to decide somehow.

# Complexity:
- Should scale linearly with respect to the total number of elements in the MxN array.
- find_center() is long in terms of number of lines of code, but is just making comparisons between variables and doing simple math, and therefore scales with O(1)
- check_positions performs similar operations also scales with O(1).
- compare_coordinates() does have a for loop, however it only cycles between the four possible directions and therefore is O(1).
- eat() contains a while loop that checks to make sure the bunny is still awake. Since the bunny's awake status depends on the number of edible squares, which could scale with the number of elements in the array, this function will scale linearly with respect to the number of elements in the array.

In [101]:
def find_center(garden_input):
    # This function takes an N x M matrix and identifies the indices of the starting cell.
    # This there are multiple centers (i.e., even number of rows and/or columns), this function
    # selects the cell with the most carrots.
    
    N_rows = len(garden_input)
    M_cols = len(garden_input[0])
    
    # If there is a center row and a center column, it is easy 
    if N_rows % 2 == 1:
        center_row = N_rows // 2
        
        if M_cols % 2 == 1:
            center_col = M_cols // 2
        
        # Center row exists, but there are odd columns:
        # Identify central column with highest carrot count in that row
        else:
            carrot_count_left = garden_input[center_row][M_cols // 2 - 1]
            carrot_count_right = garden_input[center_row][M_cols // 2]
            
            #Return indices of highest carrot cell. As per problem statement, there should not be ties.
            if carrot_count_left > carrot_count_right:
                center_col = M_cols // 2
            else:
                center_col = M_cols // 2
    else:
        # If center column exists, but there are an even number of rows: 
        # Identify central row with highest carrot count
        if M_cols % 2 == 1:
            center_col = M_cols // 2
            
            carrot_count_top = garden_input[N_rows // 2 - 1][center_col]
            carrot_count_bottom = garden_input[N_rows // 2][center_col]
            
            if carrot_count_top > carrot_count_bottom:
                center_row = N_rows // 2 - 1
            else:
                center_row = N_rows // 2
        
        #Even number of both rows and columns, find highest of the four cells
        else:
            #Define center square
            center_square = [garden_input[N_rows // 2 - 1][M_cols // 2 - 1],
                garden_input[N_rows // 2 - 1][M_cols // 2],
                garden_input[N_rows // 2][M_cols // 2 -1],
                garden_input[N_rows // 2][M_cols // 2]]
            
            #Identify max
            carrot_max = max(center_square)
            max_index = center_square.index(carrot_max)
            
            #Retrieve coordinates from the above values:
            center_row = max_index // 2 + (N_rows // 2 - 1)
            center_col = max_index % 2 + (M_cols // 2 - 1)
    return(center_row, center_col)
            
        
            

In [216]:
def check_positions(coordinate_list, garden):
    #Returns False if coordinates are not valid
    
    #Values less than zero will be off the grid
    if min(coordinate_list) < 0:
        return False
    
    #Row values greater than the number of rows will be off the grid
    elif coordinate_list[0] > len(garden) - 1:
        return False
    #Col values greater than the number of cols will be off the grid
    elif coordinate_list[1] > len(garden[0]) - 1:
        return False
    else:
        return True

In [217]:
def compare_coordinates(garden,N_position, M_position):
    # Takes the current position of bunny and looks at each of the four possible directions.
    # Returns the direction to travel.
    
    #Define directions
    top = [N_position - 1, M_position]
    bottom = [N_position + 1, M_position]
    left = [N_position, M_position - 1]
    right = [N_position, M_position + 1]
    
    #Check directions for being viable, using function defined above. 
    #Then determine the number of carrots and the direction to move.
    maximum_carrots = 0
    current_carrots = 0
    best_direction = []
    
    
    for direction in (top, bottom, left, right):
        if check_positions(direction, garden):
            current_carrots = garden[direction[0]][direction[1]]

            if current_carrots > maximum_carrots:
                maximum_carrots = current_carrots
                best_direction = direction
    
    #If there are no carrots to go to, return instructions to sleep,
    #Else, return best direction and number of carrots.
    
    if best_direction == []:
        return False, 0
    
    else:
        return best_direction, maximum_carrots

In [225]:
def eat(garden):
    
    #The bunny is awake to start
    awake_status = True
    
    #Find the center
    starting_N, starting_M = find_center(garden)
    
    
    #The bunny consumes the carrots at the starting position:
    carrot_total = int(garden[starting_N][starting_M])
    garden[starting_N][starting_M] = 0
    
    
    while awake_status:
        
        #The bunny looks around for carrots
        direction, carrots_consumed = compare_coordinates(garden, starting_N, starting_M)
        
        #If there's no carrots, bunny falls asleep
        if not direction:
            awake_status = False

        
        #Otherwise bunny eats the carrots and keeps going
        else:
            
            #Bunny moves
            starting_N = direction[0]
            starting_M = direction[1]
            
            # Eat the carrots on the square.
            garden[starting_N][starting_M] = 0
            carrot_total += carrots_consumed
            
    return carrot_total
            

# Test Function
The provided sample input is included below for convenience.

In [228]:
garden1 = [[5, 0, 8, 6, 3], [0, 0, 7, 0, 4], [4, 6, 3, 4, 9], [3, 1, 0, 5, 8]]

In [229]:
eat(garden1)

54